In [0]:
%pip install langchain-core langchain-openai langchain-community

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
open_ai_key = ""

table_address_default = "vanhack.mobi_data.silver_stations"

data_contract_default_path = "/Workspace/Users/jazz@jazzgrewal.com/mobi_agent_starter/datacontract.yml"

def read_contract(file_path):
    with open(file_path, "r") as f:
        content = f.read()

    with open(file_path, "w") as f:
        f.write(content)

    import yaml

    with open(file_path) as f:
        contract = yaml.safe_load(f)
    return contract

# data_contract_default = """
#     {
#         "table_comment": "Contains mobility station definitions for analytics V2.",
#         "columns": {
#             "station_id": "Unique identifier for the station",
#             "lat": "Latitude (WGS84)",
#             "lon": "Longitude (WGS84)"
#         }
#     }
# """

In [0]:
# Create widgets
dbutils.widgets.text("table_address", table_address_default)
dbutils.widgets.text("data_contract_path", data_contract_default_path)

# Read widget values
table_address = dbutils.widgets.get("table_address")
data_contract_path = dbutils.widgets.get("data_contract_path")

print("table_address:", table_address)
print("data_contract:", data_contract)


table_address: vanhack.mobi_data.silver_stations
data_contract: 
    {
        "table_comment": "Contains mobility station definitions for analytics V2.",
        "columns": {
            "station_id": "Unique identifier for the station",
            "lat": "Latitude (WGS84)",
            "lon": "Longitude (WGS84)"
        }
    }



In [0]:
data_contract = read_contract(data_contract_path)

In [0]:
# ------------------------------------------------------
# 0. SETUP: Load OpenAI key from Databricks Secrets
# ------------------------------------------------------
open_ai_key = open_ai_key #dbutils.secrets.get("openai", "api_key")

from openai import OpenAI
import json

client = OpenAI(api_key=open_ai_key)

def escape_table_address(table_address: str):
    parts = table_address.split(".")
    if len(parts) != 3:
        raise ValueError(f"Invalid UC table address: {table_address}")

    catalog, schema, table = parts
    return f"`{catalog}`.`{schema}`.`{table}`"


# ======================================================
# 1. METADATA UPDATER TOOL — REAL UNITY CATALOG UPDATES
# ======================================================
def metadata_creator(table_address: str, data_contract: dict):
    """
    Updates Unity Catalog table & column comments based on a data contract.

    Example contract:
    {
        "table_comment": "Table description",
        "columns": {
            "col1": "Comment1",
            "col2": "Comment2"
        }
    }
    """

    summary = []
    
    # --------------------------------------------------
    # Validate table exists
    # --------------------------------------------------
    try:
        table = spark.table(table_address)
    except Exception as e:
        return f"❌ Table not found: {table_address}\n{str(e)}"

    schema_cols = [field.name for field in table.schema.fields]


    # --------------------------------------------------
    # 1. Update table comment
    # --------------------------------------------------
    if "table_comment" in data_contract:
        comment = data_contract["table_comment"]
        escaped = escape_table_address(table_address)
        sql = f"COMMENT ON TABLE {escaped} IS '{comment}'"
        print(sql)
        spark.sql(sql)
        summary.append(f"✅ Updated table comment → {comment}")


    # --------------------------------------------------
    # 2. Update column-level comments
    # --------------------------------------------------
    if "columns" in data_contract and isinstance(data_contract["columns"], dict):

        for col, comment in data_contract["columns"].items():

            if col not in schema_cols:
                summary.append(f"⚠️ Skipped unknown column `{col}` (not in schema)")
                continue

            sql = f"COMMENT ON COLUMN {table_address}.{col} IS '{comment}'"
            spark.sql(sql)
            summary.append(f"🔹 Updated column `{col}` → {comment}")


    # --------------------------------------------------
    # 3. Done
    # --------------------------------------------------
    return "\n".join(summary)


# ======================================================
# 2. TOOL SCHEMA FOR OPENAI FUNCTION CALLING
# ======================================================
tools = [
    {
        "type": "function",
        "function": {
            "name": "metadata_creator",
            "description": "Updates table & column comments in Unity Catalog.",
            "parameters": {
                "type": "object",
                "properties": {
                    "table_address": {"type": "string"},
                    "data_contract": {
                        "type": "object",
                        "properties": {
                            "table_comment": {"type": "string"},
                            "columns": {
                                "type": "object",
                                "additionalProperties": {"type": "string"}
                            }
                        }
                    }
                },
                "required": ["table_address", "data_contract"]
            }
        }
    }
]



# ======================================================
# 3. CALL THE AGENT
# ======================================================
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a Data Governance Agent responsible for updating Unity Catalog metadata."
        },
        {
            "role": "user",
            "content": f"""
                Update metadata for this UC table:
                table_address: "{table_address}"

                Here is the data contract (JSON):
                {data_contract}
            """
        }
    ],
    tools=tools,
    tool_choice="auto"
)

msg = response.choices[0].message



# ======================================================
# 4. HANDLE TOOL CALL (Execute Python function)
# ======================================================
if msg.tool_calls:
    for call in msg.tool_calls:
        name = call.function.name
        args = json.loads(call.function.arguments)

        if name == "metadata_creator":
            result = metadata_creator(**args)
            print("=== METADATA UPDATE RESULT ===")
            print(result)

else:
    print("LLM response (no tool call):")
    print(msg.content)


COMMENT ON TABLE `vanhack`.`mobi_data`.`silver_stations` IS 'Data is intended to be used by automated and live Security Operations Center ("SOC") agents for incident analysis and action. This accelerator uses real signals from the Cavallo Technologies environment.

Limitations: Data products may be mocked to support AI & Analytics use cases. Check each table to determine whether it is mocked.

Billing: Free, MIT License.

Notice Period: P3M.

Availability: 99.9% - The accelerator is designed for high reliability, ensuring that data processing and alerting pipelines remain continuously operational except during planned maintenance windows.

Retention: Unlimited.

Latency: Data ingested from source systems is processed and made available in downstream silver and gold layers within 24 hours of arrival under normal operating conditions.

Freshness: The data product is refreshed daily to ensure that new or updated security alerts are available within 24 hours of their source event time.

Su